**Author:** Beatrice Occhiena s314971. See [`LICENSE`](https://github.com/beatrice-occhiena/Computational_intelligence/blob/main/LICENSE) for details.
- institutional email: `S314971@studenti.polito.it`
- personal email: `beatrice.occhiena@live.it`
- github repository: [https://github.com/beatrice-occhiena/Computational_intelligence.git](https://github.com/beatrice-occhiena/Computational_intelligence.git)

**Resources:** These notes are the result of additional research and analysis of the lecture material presented by Professor Giovanni Squillero for the Computational Intelligence course during the academic year 2023-2024 @ Politecnico di Torino. They are intended to be my attempt to make a personal contribution and to rework the topics covered in the following resources.
- [https://github.com/squillero/computational-intelligence](https://github.com/squillero/computational-intelligence)
- Stuart Russel, Peter Norvig, *Artificial Intelligence: A Modern Approach* [3th edition]

.

.


# Problem Solving by Searching
---

## Why search?

> Search techniques are utilized in problem-solving scenarios when there is a clear, predefined goal that needs to be achieved, and the problem can be logically broken down into different states and actions.

Search algorithms are essential for tackling problems in various domains and are especially useful in cases when:
- The problem's **complexity** is beyond straightforward mathematical modeling or doesn't have a known analytical solution.
- We **lack prior knowledge** of the exact sequence of actions leading to a solution.
- We intend to systematically explore a **large solution space**, requiring an examination of various paths to find the best one.
- We seek **optimal or near-optimal solutions** in extensive search spaces.

### Problem solving agent
A problem-solving agent is a type of goal-based agent capable of determining a sequence of actions leading to a desired goal. This agent achieves this by exploring the problem's state space, which encompasses all potential states reachable from the initial state through a sequence of actions.

The agent's program comprises two key components:
- **Problem & Goal Formulation** This entails the process of deciding which actions and states to consider based on the specified goal. 🗺️🎯..
- **Search Algorithm** This involves determining the next action to take given the current state in order to progress toward the goal. 🧭..

### Conditions for solving problems by searching
While path search algorithms can be applied to a wide range of problems, there are some conditions and considerations that make them more suitable for certain types of problems.

When the task environment is
- discrete
- fully observable
- deterministic
- static
- completelly known

> In this case, the agent can be said to have `full control` of the environment, therefore it can plan ahead and choose a sequence of actions that will lead to the desired goal. The process of finding a sequence of actions is called *search*.

## Problem definition

A problem can be formally defined by the following components:
1. **Initial state** The state in which the agent begins. $s_0$
2. **Set of actions** The set of actions available to the agent in each state. $\mathcal{A}(s)=\{a_1, a_2, \dots, a_n\}$
3. **Transition model** A description of what each action does. $\mathcal{S}(s,a)=s'$
4. **Goal test** A function that determines whether a given state is a goal state. $\mathcal{G}(s)$
5. **Path cost** A function that assigns a numeric cost to each path. It is assumed that the cost of a path is the sum of the costs of its actions, i.e. the sum of each step's cost. $g(s_0, a_1, \dots, a_n)=\sum_{i=1}^n c(s_i, a_i, s_{i+1})$

### State space
The state space is the set of all states reachable from the initial state by any sequence of actions. It is denoted by $\mathcal{S}$. The state space is a **directed graph**, where 
- the nodes represent `states` ⭕..
- the edges represent `actions` ➡️..

### Searching for solutions
A solution to a problem is a sequence of actions that leads *from the initial state to a goal state*. A solution is `optimal` if it has the lowest path cost among all solutions.

Depending on the way we want our search algorithm to explore the state space, we can distinguish between two types of search.

##### 1 - 🌳 Tree search 🌳
In a tree search, the search algorithm explores a search tree without considering whether it has visited a state before. It doesn't keep track of the states it has already explored.

$\implies$ different nodes can represent the same state.

The initial state of the problem serves as the `root` node of the tree. As the search algorithm progresses, it `expands nodes` by considering possible actions and generating child nodes for each state that can be reached.
- `o` `Memory usage`: It tends to use less memory because it only maintains the current tree and doesn't remember visited states.
- `x` `Redundancy`: It may explore the same state multiple times, leading to inefficiency.
- `x` `Un-completeness`: It may not find a solution even if one exists, due to infinite loops in a space with cycles.

Tree search is typically used in cases where memory resources are limited and revisiting states isn't an issue. It's suitable for cases where the solution is guaranteed to be found quickly or where the state space doesn't contain cycles.
  
##### 2 - 📊 Graph search 📊
In graph search, the search algorithm keeps track of the states it has already explored, ensuring that it doesn't revisit them. This is crucial for practical problem-solving because revisiting states can be inefficient and may lead to infinite loops.

$\implies$ nodes represent unique states.

- `x` `Memory usage`: It tends to use more memory because it needs an additional data structure to keep track of visited states.
- `o` `Efficient exploration`: In graph search, the exploration extends across the entire state space, keeping track of visited states to avoid revisiting them. It maintains a more comprehensive record of the explored states.
- `o` `Completeness`: It is guaranteed to find a solution if one exists, as it doesn't get stuck in infinite loops.

Graph search is used when memory resources are more abundant, and the state space may contain cycles or when it's essential to guarantee that a solution, if it exists, will be found.

## Data structures, functions and terminology

### State space
In search problems, agents treat states and actions as `atomic` entities. This means that they don't need to know the internal structure of states and actions, but only how to compare them and how to apply actions to states.

Depending on the problem, the state space can be represented as a graph, a matrix, or any appropriate data structure.

> This is where we need to unleash all our inventiveness and creativity to come up with a representation that allows us to effectively represent every aspect of the problem.

Our representation has to:
- exhaustive = contain all the information necessary to describe the problem
- abstract = be as compact as possible, to avoid wasting memory resources

In [6]:
# Example: 8-puzzle problem
initial_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

### Actions
Actions are the atomic entities that agents can perform to change the state of the environment. They are usually implemented by two functions:
- `actions(state)` returns the set of actions available in a given state.
- `apply_action(state, action)` returns the state that results from applying an action to a given state.

In [ ]:
# Example: 8-puzzle problem
def actions(state):
    actions_list = []
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                if i > 0:
                    actions_list.append('up')
                if i < 2:
                    actions_list.append('down')
                if j > 0:
                    actions_list.append('left')
                if j < 2:
                    actions_list.append('right')
    return actions_list

In [ ]:
# Example: 8-puzzle problem
def apply_action(state, action):
    # Create a copy of the current state to avoid modifying the original state
    new_state = [row[:] for row in state]

    # Find the row and column of the blank (0) tile
    for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    empty_row, empty_col = i, j

    # Perform the action by swapping the blank tile and an adjacent tile
    if action == "up":
        new_state[empty_row][empty_col], new_state[empty_row - 1][empty_col] = (new_state[empty_row - 1][empty_col], new_state[empty_row][empty_col],)
    elif action == "down":
        new_state[empty_row][empty_col], new_state[empty_row + 1][empty_col] = (new_state[empty_row + 1][empty_col], new_state[empty_row][empty_col],)
    elif action == "left":
        new_state[empty_row][empty_col], new_state[empty_row][empty_col - 1] = (new_state[empty_row][empty_col - 1], new_state[empty_row][empty_col],)
    elif action == "right":
        new_state[empty_row][empty_col], new_state[empty_row][empty_col + 1] = (new_state[empty_row][empty_col + 1], new_state[empty_row][empty_col],)

    return new_state

### Frontier
The set of all *nodes available for expansion at any given point* is called the frontier. These nodes have been generated but not yet expanded.
- **init** The frontier will initially contain only the initial state.
- **loop** The process of expanding nodes on the frontier continues until 
  - 🎯 a goal state is found 
  - 🚫 there are no more nodes to expand => no solution exists
  - ✋🏻 a predefined limit is reached

It is implemented as a `queue` in which the order of the nodes to be extracted is determined by the search strategy.
- FIFO
- LIFO
- Priority

🧍🏻🧍🏻‍♂️🧍🏻‍♀️🧍🏻🧍🏻‍♂️🧍🏻‍♀️... Next!

### Explored set
📊! The explored set is a data structure that keeps track of the *already visited states*. And its presence is what distinguishes graph search from tree search!

It is implemented as a `set` or `hash table` to facilitate the search for visited states.

🧍🏻‍♀️✋🏻.. Not you again! 🧍🏻🧍🏻‍♂️🧍🏻‍♀️🧍🏻🧍🏻‍♂️🧍🏻‍♀️

### Expanding a node
When a node is `extracted` from the frontier, it is `expanded` by generating its child nodes.
- **generate** Children are obtained by applying each possible action to the current state.
- **add** The newly generated successor nodes are added to the frontier.

📊! For graph search, the newly generated nodes are only added to the frontier if they haven't been visited before. They are checked against the `explored set` to ensure that they are unique.

### Goal check
When a node is extracted from the frontier, it is checked to see if it is a goal state. If it is, the search is terminated and the solution is returned. Otherwise, the search continues.

The check is implemented as a `function` that takes a state as input and returns a boolean value. 

🎯🔍???..

In [ ]:
# Example: 8-puzzle problem
def goal_check(state):
    return state == [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

In [ ]:
# Example: 8-puzzle problem - graph search
from queue import SimpleQueue

# frontier
frontier = SimpleQueue()
frontier.put(initial_state)

# explored set
explored = set()

while not frontier.empty():
    # get the next state from the frontier
    curr_state = frontier.get()
    explored.add(curr_state)

    # check if the current state is the goal state
    if goal_check(curr_state):
        print('Found!')
        break

    # add the new states to the frontier
    for action in actions(curr_state):
        new_state = apply_action(curr_state, action)
        if new_state not in explored:
            frontier.put(new_state)

### Path reconstruction, path cost & other node related information
When the goal state is not representative of the solution (i.e. the sequence of actions needed to solve the problem), the path from the initial state to the goal state can be reconstructed in various ways. 

The total cost of the path is strictly related to the same principle, as it is the sum of the costs of each step in the path. And even if usually the step cost is the same for all actions, in some cases it may vary depending on the state or action.

Other information about the node can be stored in the node itself. For example, the depth of the node in the tree, the action that led to the node, etc.

To tackle these issues, we can use different approaches such as:

- **parent pointers** In graph search, in which we don't want to revisit states, each node can contain a pointer to its only parent node. The path can be reconstructed by following the parent pointers of each node. This can be implemented by:
  - a `dictionary` in which the key is the child node and the value is the parent node
  - a `function` that takes the goal state as input and returns the list of states/actions leading to it
- **recursive search** When using recursion for tree search, you would typically implement a recursive function that explores child nodes from a given state, and as it explores deeper into the tree, it can keep track of the parent-child relationships. The information about the path in this case is stored in the call stack, but keep in mind that for very deep trees, recursion may lead to stack overflow.
- **node representation** In tree search, nodes and states do not coincide. Since I can guarantee that each node has a unique predecessor, I can store the path from the initial state to the goal only when I create a wrapper class for the nodes. This can be implemented by:
  - a `class Node` that contains the state, the parent node, the path cost and other information about the node (e.g. depth, action, etc.)
- **visual representation** In tree search, in which a state can have multiple parent nodes, we can find an inventive way to visually represent the path from the initial state to the goal state. This can be implemented by:
  - a `graph` in which storing all possible node-states
  - a `function` that adds an edge between the current node and its parent node
  - a `function` to visualize the graph

  (See https://github.com/squillero/computational-intelligence/blob/master/2023-24/4-friends.ipynb for an example)

> Reconstructing the path surely can be a challenging memory-intensive task, but fortunately, it is not always necessary. In some cases, the goal state can be representative of the solution, and the path can be reconstructed by following the parent pointers of each node. In others we're only interested in finding if a solution exists, and not in the path itself.

🐜...🐜..🐜.....🐜

In [ ]:
# Example: Node class
class Node:
    def __init__(self, state, parent, action, depth, cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        self.cost = cost
        self.is_expanded = False
    
    def expand(self):
        actions_list = actions(self.state)
        children = []
        for action in actions_list:
            child_state = apply_action(self.state, action)
            child = Node(child_state, self, action, self.depth + 1, self.cost + 1)
            children.append(child)
        self.is_expanded = True
        return children

## Search strategies

### Measuring search performance

### Uninformed search strategies

#### Breadth-first search (BFS)

#### Depth-first search (DFS)

#### Uniform-cost search (UCS)

#### Depth-limited search (DLS)

#### Iterative deepening search (IDS)

#### Bidirectional search (BDS)

#### Beam search

### Informed search strategies

#### Greedy best-first search (GBFS)

#### A* search


- search tree vs search graph: data structures
  => treat states and actions as **atomic** entities

## Search strategies
Their performance is measured in terms of:
- Completeness: is the algorithm guaranteed to find a solution when there is one?
- Optimality: does the strategy find the best solution?
- Time complexity: how long does it take to find a solution?
- Space complexity: how much memory is needed to perform the search?
Complexity depends on:
- b: maximum branching factor of the search tree
- d: depth of the least-cost solution
 



